In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
sns.set()

In [2]:
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightEdges = BabyFlightNodes.read().split("%SECTION%")[2]
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightNodes = BabyFlightNodes.read().split("%SECTION%")[1]

In [3]:
with open('Flight2.txt', mode='r') as BabyFlight:
    Flight1 = BabyFlight.read().split("\n")[0]

In [4]:
from scipy import spatial
from ipynb.fs.full.GetTrajectory import GetTrajectory
from ipynb.fs.full.PerpendicularDistance import NearestPointOnLine
from ipynb.fs.full.PerpendicularDistance import GetPerpendicularDistance
from ipynb.fs.full.PerpendicularDistance import EdgeSquareSearch, EdgeCircleSearch
from ipynb.fs.full.PerpendicularDistance import Haversine, FilterTrajectory, EdgeCovered, GetDirection1, GetHeading, GetAlpha, GetBeta, GetDistanceChange, GetHeadingIncrement, GetDirection2
from ipynb.fs.full.PerpendicularDistance import GetEdgesBetween, GetEdgeDistance, GetCorrectedEdge, GetCorrectedEdge2, GetEdgeDistance2, GetDistanceError, GetNeighbor, GetConnectivity, IsSublist


In [5]:
FlightTrajectoryNodes = GetTrajectory(Flight1)


In [6]:
from ipynb.fs.full.GetTaxiway import GetTaxiway
from ipynb.fs.full.GetTaxiway import GetEdgeID
from ipynb.fs.full.GetTaxiway import GetEdgeStart
from ipynb.fs.full.GetTaxiway import GetEdgeEnd
from ipynb.fs.full.GetTaxiway import GetEdgeLength
from ipynb.fs.full.GetTaxiway import GetEdgeSpecification
from ipynb.fs.full.GetTaxiway import GetEdgesAndNodes, GetNodeLinks

In [7]:
from ipynb.fs.full.FuzzyLogic import SMP1FuzzyLogic
from ipynb.fs.full.FuzzyLogic import SMP2FuzzyLogic


In [8]:
#Setting up some lists to sort through the data
FlightNodesList = GetTaxiway(FlightNodes, FlightEdges)[0]
FlightEdgesList = GetTaxiway(FlightNodes, FlightEdges)[1]
FlightEdgesWeights = GetTaxiway(FlightNodes, FlightEdges)[2]
AllNodeLinks = GetNodeLinks(FlightNodesList, FlightEdgesList)

In [9]:
EdgesAndNodes = GetEdgesAndNodes(FlightEdgesList, FlightNodesList)
#EdgeID[0], NodeStart[1], NodeFinish[2], EdgeLength[3], EdgeSpecification[4], StartX[5], StartY[6], EndX[7], EndY[8]


In [10]:
#Making arrays of lists for easier searching
GateEdges = []
for item in EdgesAndNodes:
    if item[4] == 'gate':
        GateEdges.append(item)

GateNodes = []
Node = []
Nodes = []
for item in FlightNodesList:
    Nodes.append((float(item[1]),float(item[2])))
for item in FlightTrajectoryNodes:
    Node.append((float(item[0]),float(item[1])))
for item in GateEdges:
    GateNodes.append((item[5], item[6]))
    GateNodes.append((item[7], item[8]))
    
tree = spatial.KDTree(Nodes) #This uses spatial from scipy


In [11]:
G = nx.Graph()

Nodict = {}
for index, item in enumerate(Node):
    NodeX = item[0]
    NodeY = item[1]
    Nodict[int(index+1)] = (NodeX, NodeY)
    
MapNodes = {}

for item in FlightNodesList:
    NodeX = float(item[1])
    NodeY = float(item[2])
    MapNodes[int(item[0])] = (NodeX, NodeY)

G.add_nodes_from(MapNodes.keys())
for n, p in iter(MapNodes.items()):
    G.node[n]['pos'] = p

NodeLinks = []
for item in EdgesAndNodes:
    NodeLinks.append((item[1], item[2], {'weight':item[3]}))
G.add_edges_from(NodeLinks)

#specification = nx.get_edge_attributes(G, 'specification')
weight = nx.get_edge_attributes(G, 'weight')
pos = nx.get_node_attributes(G, 'pos')

In [12]:
Node = FilterTrajectory(Node, 22.3, 113.9185, 0.1)
Node

[(22.334335, 114.01472),
 (22.33232, 114.00869),
 (22.330536, 114.00333),
 (22.328222, 113.996735),
 (22.326187, 113.99114),
 (22.324219, 113.98565),
 (22.322159, 113.979706),
 (22.319687, 113.97207),
 (22.317888, 113.96647),
 (22.316345, 113.96163),
 (22.313139, 113.951515),
 (22.311768, 113.94705),
 (22.310299, 113.94226),
 (22.308718, 113.93717),
 (22.30732, 113.93255),
 (22.30583, 113.927765),
 (22.303457, 113.920494),
 (22.301788, 113.91534),
 (22.30037, 113.91094),
 (22.29918, 113.90725),
 (22.298172, 113.904205),
 (22.297405, 113.90177),
 (22.296844, 113.90001),
 (22.296535, 113.89907),
 (22.296461, 113.89883),
 (22.296333, 113.89848),
 (22.296438, 113.89816),
 (22.296764, 113.897934),
 (22.297016, 113.897835),
 (22.297264, 113.89774),
 (22.297497, 113.89765),
 (22.297718, 113.89757),
 (22.297987, 113.89782),
 (22.298138, 113.8983),
 (22.29823, 113.8986),
 (22.298332, 113.898895),
 (22.2984, 113.89911),
 (22.298481, 113.8994),
 (22.298603, 113.89974),
 (22.298754, 113.90024),
 (

In [13]:
#Check which end has a gate nearest
#Check The Start using KDTree array find the closest Gate Nodes to the trajectory node
tree = spatial.KDTree(GateNodes)
HighlightedGateNodeIDs = list(tree.query(Node[0], 5)[1])
HighlightedStartGateNodes = []
for n in HighlightedGateNodeIDs:
        HighlightedStartGateNodes.append(GateNodes[n])

PointX = Node[0][0]
PointY = Node[0][1]
HighlightStartGateEdge = []
for m in GateEdges:
        for i in HighlightedStartGateNodes:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightStartGateEdge.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightStartGateEdge.append(m)
        else:
            pass
HighlightedStartGateEdges = list(set(HighlightStartGateEdge))
StartGateEdgesSuccess = []

for index, edge in enumerate(HighlightedStartGateEdges):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.001) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            print(edge)
            print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance)
            StartGateEdgesSuccess.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance))
            ChosenStartEdge = min(StartGateEdgesSuccess, key = lambda t: t[-1])
        else:
            print('This gate edge is not near')

#Check the end
HighlightedGateNodeIDs = list(tree.query(Node[-1], 5)[1])
HighlightedEndGateNodes = []
for n in HighlightedGateNodeIDs:
        HighlightedEndGateNodes.append(GateNodes[n])  
PointX = Node[-1][0]
PointY = Node[-1][1]
HighlightEndGateEdge = []
for m in GateEdges:
        for i in HighlightedEndGateNodes:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightEndGateEdge.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightEndGateEdge.append(m)
        else:
            pass

HighlightedEndGateEdges = list(set(HighlightEndGateEdge))

EndGateEdgesSuccess = []
for index, edge in enumerate(HighlightedEndGateEdges):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.001) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            print(edge)
            print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance)
            EndGateEdgesSuccess.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance))
            ChosenEndEdge = min(EndGateEdgesSuccess, key = lambda t: t[-1])
        else:
            print('This gate edge is not near')

#Reverse the Trajectory Nodes if a gate is found at the end
try:
    if ChosenEndEdge == []:
        StartEdge = ChosenStartEdge
    else:
        pass
except NameError:
    StartEdge = ChosenStartEdge
    ChosenEndEdge = []
try:
    if ChosenStartEdge == []:
        StartEdge = ChosenEndEdge
    else:
        pass
except NameError:
    StartEdge = ChosenEndEdge
    ChosenStartEdge = []
try:
    if ChosenEndEdge[-1] > ChosenStartEdge[-1]:
        StartEdge = ChosenStartEdge
except IndexError:
    pass
try:
    if ChosenStartEdge[-1] > ChosenEndEdge[-1]:
        StartEdge = ChosenEndEdge
except IndexError:
    pass

if StartEdge == ChosenEndEdge:
    NewNode = list(reversed(Node))
else:
    NewNode = Node

#If neither start nor finish are near a gate, then choose closest Edge

#Search for 5 closest EdgeNodes to Node[0]
if ChosenStartEdge == [] and ChosenEndEdge == []:
        tree = spatial.KDTree(Nodes)
        HighlightedNodeIDs = list(tree.query(NewNode[0], 5)[1])
        HighlightedNodes = []
        for n in HighlightedNodeIDs:
            HighlightedNodes.append(Nodes[n])  

        HighlightEdge = []
        for m in EdgesAndNodes:
            for i in HighlightedNodes:
                if m[5] == i[0] and m[6] == i[1]:
                    HighlightEdge.append(m)
                elif m[7] == i[0] and m[8] == i[1]:
                    HighlightEdge.append(m)
            else:
                pass
        HighlightedEdges = list(set(HighlightEdge))



        #Iterate through the Edges for each Trajectory Node
        for index, edge in enumerate(HighlightedEdges):
            EdgeID = edge[0]
            EdgeStart = edge[1]
            EdgeEnd = edge[2]
            EdgeLength = edge[3]
            EdgeType = edge[4]

            StartX = edge[5]
            StartY = edge[6]
            EndX = edge[7]
            EndY = edge[8]
            #If Edge is within radius r0 of Node, then return True
            if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.001) == True:
            #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
                (PrevIntX, PrevIntY) = NearestPointOnLine(PrevX, PrevY, StartX, StartY, EndX, EndY)
                (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
                PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
                (NewStartX, NewStartY, NewEndX, NewEndY) = GetDirection2(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY)
                print(NewStartX, NewStartY, NewEndX, NewEndY)
                Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
                if counter == 0:
                    pass
                print(edge)
                print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance, Percentage)
                EndGateEdgesSuccess.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance))
                ChosenEdge = min(EndGateEdgesSuccess, key = lambda t: t[-1])
            else:
                pass
            StartEdge = ChosenEdge
else:
    pass

This gate edge is not near
This gate edge is not near
This gate edge is not near
This gate edge is not near
This gate edge is not near
(2543, 2291, 2292, 135.0328633602965, 'gate', 22.3134847, 113.9302818, 22.3123401, 113.930723)
0 2543 22.3134847 113.9302818 22.3123401 113.930723 (22.313463787569095, 113.93028986095102) 0.0006750337927323224
(2496, 2209, 2210, 132.090974445492, 'gate', 22.3141092, 113.9293734, 22.3152271, 113.9289365)
1 2496 22.3141092 113.9293734 22.3152271 113.9289365 (22.3141092, 113.9293734) 0.0009332945944347705
(2542, 2289, 2290, 137.72245475253408, 'gate', 22.3132837, 113.9296113, 22.3121182, 113.930067)
2 2542 22.3132837 113.9296113 22.3121182 113.930067 (22.31321279783337, 113.92963902210839) 2.2524375102628767e-05
(2541, 2287, 2288, 137.67889012093454, 'gate', 22.3130554, 113.9289541, 22.3119587, 113.9295773)
3 2541 22.3130554 113.9289541 22.3119587 113.9295773 (22.312877362096234, 113.92905527007534) 0.0006955467566478238


In [14]:
StartEdge

(2542,
 2289,
 2290,
 137.72245475253408,
 'gate',
 22.3132837,
 113.9296113,
 22.3121182,
 113.930067,
 (22.31321279783337, 113.92963902210839),
 2.2524375102628767e-05)

In [15]:
#The Algorithm
#Filter Nodes outside of the airport...

#Which way should we iterate through points? Must start at a gate and end at a gate or runway... Check if gate is at start or end

#Once first edge is found... Follow along a path with PrevMatchedEdge as a variable, and is current edge a junction?

#If Current Edge is a junction, initiate second Fuzzy rules
FinalNode = []
EdgePath = [StartEdge]            
for counter, item in enumerate(NewNode):
    print('\n\n')
    print((counter, item,'NEW NODE'))
    
    #At the start or end, a Gate must be present
    if counter == 0:
        continue
    else:
        PrevNode = NewNode[counter -1]
        NextNode = NewNode[counter +1]
        Next2Node = NewNode[counter +2]
    PrevX = PrevNode[0]
    PrevY = PrevNode[1]
    PointX = item[0]
    PointY = item[1]
    NextX = NextNode[0]
    NextY = NextNode[1]
    Next2X = Next2Node[0]
    Next2Y = Next2Node[1]
    
    #Following along a link - SMP1: Does the current Node link with the previous edge?
    EdgeID = StartEdge[0]
    EdgeStart = StartEdge[1]
    EdgeEnd = StartEdge[2]
    EdgeLength = StartEdge[3]
    EdgeType = StartEdge[4]
        
    StartX = StartEdge[5]
    StartY = StartEdge[6]
    EndX = StartEdge[7]
    EndY = StartEdge[8]
    print(PrevX, PrevY, StartX, StartY, EndX, EndY)
    (PrevIntX, PrevIntY) = NearestPointOnLine(PrevX, PrevY, StartX, StartY, EndX, EndY)
    (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
    (PrevStartX, PrevStartY, PrevEndX, PrevEndY, Direction) = GetDirection2(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY)
    if Direction == True:
        PrevEdgeStart = EdgeStart
        PrevEdgeEnd = EdgeEnd
    else:
        PrevEdgeStart = EdgeEnd
        PrevEdgeEnd = EdgeStart
    #Input Variables:
    Alpha = GetAlpha(PrevX, PrevY, PointX, PointY, PrevStartX, PrevStartY, PrevEndX, PrevEndY)
    Beta = GetBeta(PointX, PointY, PrevStartX, PrevStartY, PrevEndX, PrevEndY)
    DeltaDistance = GetDistanceChange(PrevX, PrevY, PointX, PointY, PrevStartX, PrevStartY, PrevEndX, PrevEndY)
    HeadingIncrement = GetHeadingIncrement(PrevX, PrevY, PointX, PointY, NextX, NextY)
    Z = SMP1FuzzyLogic(HeadingIncrement, HeadingIncrement, DeltaDistance, Alpha, Beta)
    print('ALPHA', Alpha,'BETA', Beta,'D', DeltaDistance,'Theta', HeadingIncrement, 'Z', Z)
    if Z > 30:
        FinalNode.append((IntX, IntY))
        if EdgePath[-1][0] == StartEdge[0]:
            print('PREVIOUS EDGE CHOSEN', StartEdge)
        else:
            EdgePath.append(StartEdge)
        continue
    #First Fuzzy Logic to check if current node matches with
    
    #If SMP-1 states it has passed previous Edge...
    #Search for 8 closest EdgeNodes to Node and highlight the Edges connected to them
    tree = spatial.KDTree(Nodes)
    HighlightedNodeIDs = list(tree.query(item, 15)[1])
    HighlightedNodes = []
    for n in HighlightedNodeIDs:
        HighlightedNodes.append(Nodes[n])  
    
    HighlightEdge = []
    for m in EdgesAndNodes:
        for i in HighlightedNodes:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightEdge.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightEdge.append(m)
        else:
            pass
    HighlightedEdges = list(set(HighlightEdge))

    
    #Iterate through the Edges for each Trajectory Node
    EdgesSuccess = []
    for index, edge in enumerate(HighlightedEdges):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.0015) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (PrevIntX, PrevIntY) = NearestPointOnLine(PrevX, PrevY, StartX, StartY, EndX, EndY)
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            HaversineDistance = Haversine(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            
            #Check if Edge is being traversed in reverse or not
            (NewStartX, NewStartY, NewEndX, NewEndY, Direction) = GetDirection2(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY)
            if Direction == True:
                NewEdgeStart = EdgeStart
                NewEdgeEnd = EdgeEnd
            else:
                NewEdgeStart = EdgeEnd
                NewEdgeEnd = EdgeStart
            
            print('NEW EDGE', edge)
            print('HAVERSINE', HaversineDistance*1000) 
            Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
            if counter == 0:
                pass
            
            
            #Distance Error
            NodePath = nx.dijkstra_path(G, PrevEdgeEnd, edge[1])
            print(NodePath)
            DistanceError = GetDistanceError(NodePath, NewEdgeEnd, PrevX, PrevY, PrevIntX, PrevIntY, PrevEndX, PrevEndY, PointX, PointY, IntX, IntY, EdgesAndNodes, StartEdge, edge, FlightNodesList)
            print(DistanceError, 'Distance Error!', NewEdgeEnd, 'NewEdgeEnd')
            
            #Connectivity
            Connectivity = GetConnectivity(PrevEdgeEnd, edge, AllNodeLinks)
            print(Connectivity, 'Connectivity', PrevEdgeEnd, 'PrevEdgeEnd')
            #SMP2 Fuzzy Logic - Deciding the junction
            SMP2 = SMP2FuzzyLogic(Connectivity, DistanceError, HaversineDistance*1000)
            EdgesSuccess.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2))
            print(EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2)
            
        else:
            print('EDGE DISCARDED')
            pass
    #GET CONSEQUENT NODE HIGHLIGHTED EDGES
    HighlightedNodeIDs2 = list(tree.query(NextNode, 15)[1])
    HighlightedNodes2 = []
    for n in HighlightedNodeIDs2:
        HighlightedNodes2.append(Nodes[n])  
    
    HighlightEdge2 = []
    for m in EdgesAndNodes:
        for i in HighlightedNodes2:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightEdge2.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightEdge2.append(m)
        else:
            pass
    HighlightedEdges2 = list(set(HighlightEdge2))

    #CHECK THE CONSEQUENT NODE for the same Junction
    print('CHECKING THE NEXT NODE', '\n')
    EdgesSuccess2 = []
    for index, edge in enumerate(HighlightedEdges2):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(NextX, NextY, StartX, StartY, EndX, EndY, 0.0015) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (PrevIntX, PrevIntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            (IntX, IntY) = NearestPointOnLine(NextX, NextY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(NextX, NextY, (IntX, IntY)[0], (IntX, IntY)[1])
            HaversineDistance = Haversine(NextX, NextY, (IntX, IntY)[0], (IntX, IntY)[1])
            
            #Check if Edge is being traversed in reverse or not
            (NewStartX, NewStartY, NewEndX, NewEndY, Direction) = GetDirection2(PointX, PointY, NextX, NextY, StartX, StartY, EndX, EndY)
            if Direction == True:
                NewEdgeStart = EdgeStart
                NewEdgeEnd = EdgeEnd
            else:
                NewEdgeStart = EdgeEnd
                NewEdgeEnd = EdgeStart
            
            Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
            if counter == 0:
                pass
            
            #Distance Error
            NodePath = nx.dijkstra_path(G, PrevEdgeEnd, edge[1])
            DistanceError = GetDistanceError(NodePath, NewEdgeEnd, PointX, PointY, PrevIntX, PrevIntY, PrevEndX, PrevEndY, NextX, NextY, IntX, IntY, EdgesAndNodes, StartEdge, edge, FlightNodesList)
            
            #Connectivity
            Connectivity = GetConnectivity(PrevEdgeEnd, edge, AllNodeLinks)
            print(Connectivity, 'Connectivity', PrevEdgeEnd, 'PrevEdgeEnd')
            #SMP2 Fuzzy Logic - Deciding the junction
            SMP2 = SMP2FuzzyLogic(Connectivity, DistanceError, HaversineDistance*1000)
            EdgesSuccess2.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2))
        else:
            print('EDGE DISCARDED')
            pass

    StartEdge1 = max(EdgesSuccess, key = lambda t: t[-1])
    StartEdge2 = max(EdgesSuccess2, key = lambda t: t[-1])
    
    if StartEdge1[0] == StartEdge2[0]:
        StartEdge = StartEdge1
    
    else:
        print('CHECKING THE NEXTNEXT NODE', '\n')
        #GET NEXT NEXT NODE HIGHLIGHTED EDGES
        HighlightedNodeIDs3 = list(tree.query(Next2Node, 15)[1])
        HighlightedNodes3 = []
        for n in HighlightedNodeIDs3:
            HighlightedNodes3.append(Nodes[n])  

        HighlightEdge3 = []
        for m in EdgesAndNodes:
            for i in HighlightedNodes3:
                if m[5] == i[0] and m[6] == i[1]:
                    HighlightEdge3.append(m)
                elif m[7] == i[0] and m[8] == i[1]:
                    HighlightEdge3.append(m)
            else:
                pass
        HighlightedEdges3 = list(set(HighlightEdge3))

            #Check the next next node
        EdgesSuccess3 = []
        for index, edge in enumerate(HighlightedEdges3):
            EdgeID = edge[0]
            EdgeStart = edge[1]
            EdgeEnd = edge[2]
            EdgeLength = edge[3]
            EdgeType = edge[4]

            StartX = edge[5]
            StartY = edge[6]
            EndX = edge[7]
            EndY = edge[8]
            #If Edge is within radius r0 of Node, then return True
            if EdgeCircleSearch(Next2X, Next2Y, StartX, StartY, EndX, EndY, 0.0015) == True:
            #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
                (PrevIntX, PrevIntY) = NearestPointOnLine(NextX, NextY, StartX, StartY, EndX, EndY)
                (IntX, IntY) = NearestPointOnLine(Next2X, Next2Y, StartX, StartY, EndX, EndY)
                PDistance = GetPerpendicularDistance(Next2X, Next2Y, (IntX, IntY)[0], (IntX, IntY)[1])
                HaversineDistance = Haversine(Next2X, Next2Y, (IntX, IntY)[0], (IntX, IntY)[1])

                #Check if Edge is being traversed in reverse or not
                (NewStartX, NewStartY, NewEndX, NewEndY, Direction) = GetDirection2(NextX, NextY, Next2X, Next2Y, StartX, StartY, EndX, EndY)
                if Direction == True:
                    NewEdgeStart = EdgeStart
                    NewEdgeEnd = EdgeEnd
                else:
                    NewEdgeStart = EdgeEnd
                    NewEdgeEnd = EdgeStart

                Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
                if counter == 0:
                    pass

                #Distance Error
                NodePath = nx.dijkstra_path(G, PrevEdgeEnd, edge[1])
                DistanceError = GetDistanceError(NodePath, NewEdgeEnd, NextX, NextY, PrevIntX, PrevIntY, PrevEndX, PrevEndY, Next2X, Next2Y, IntX, IntY, EdgesAndNodes, StartEdge, edge, FlightNodesList)

                #Connectivity
                Connectivity = GetConnectivity(PrevEdgeEnd, edge, AllNodeLinks)
                print(Connectivity, 'Connectivity', PrevEdgeEnd, 'PrevEdgeEnd')
                #SMP2 Fuzzy Logic - Deciding the junction
                SMP2 = SMP2FuzzyLogic(Connectivity, DistanceError, HaversineDistance*1000)
                EdgesSuccess3.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2))
            else:
                print('EDGE DISCARDED')
                pass
        try:
            StartEdge3 = max(EdgesSuccess3, key = lambda t: t[-1])
        except ValueError:
            break
            
        Path1 = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge1[1])
        Path2 = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge2[1])
        Path3 = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge3[1])
        if IsSublist(Path2, Path1) == True and IsSublist(Path3, Path2) == True:
            StartEdge = StartEdge1
        elif IsSublist(Path3, Path2) == True and IsSublist(Path2, Path1) == False:
            StartEdge = StartEdge2
        elif IsSublist(Path2, Path1) == True and IsSublist(Path3, Path2) == False:
            StartEdge = StartEdge2
        else:
            StartEdge = StartEdge3
    PathSegment = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge[1])
    print(PathSegment)
    print(StartEdge, 'STARTEDGE')
    (NewIntX, NewIntY) = NearestPointOnLine(PointX, PointY, StartEdge[5], StartEdge[6], StartEdge[7], StartEdge[8])
    FinalNode.append((NewIntX, NewIntY))
    print(StartEdge, 'START EDGE', EdgePath[-1], 'LAST EDGE')
    if EdgePath[-1][0] == StartEdge[0]:
        print('PREVIOUS EDGE CHOSENNN')
    else:
        EdgePath.append(StartEdge)
        
    
    




(0, (22.313221, 113.92966), 'NEW NODE')



(1, (22.312866, 113.92977), 'NEW NODE')
22.313221 113.92966 22.3132837 113.9296113 22.3121182 113.930067
ALPHA 4.138977059476417 BETA 0.30619404363940106 D 0.08853717023764152 Theta 17.522551692357922 Z 35.923723115487356
PREVIOUS EDGE CHOSEN (2542, 2289, 2290, 137.72245475253408, 'gate', 22.3132837, 113.9296113, 22.3121182, 113.930067, (22.31321279783337, 113.92963902210839), 2.2524375102628767e-05)



(2, (22.312592, 113.92996), 'NEW NODE')
22.312866 113.92977 22.3132837 113.9296113 22.3121182 113.930067
ALPHA 13.383574632881505 BETA 8.62921030849679 D 0.05240386282397193 Theta 26.218353400848855 Z 24.152249379982145
NEW EDGE (1402, 1504, 764, 9.450536683575947, 'taxiway', 22.3120181, 113.9308062, 22.3119335, 113.9308156)
HAVERSINE 107.96435379868977
[2290, 775, 773, 764, 1504]
147.6920160096607 Distance Error! 764 NewEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
1402 1504 764 9.450536683575947 taxiway 22.3120181 113.9308062 22.3119335 113.9308

0 Connectivity 2290 PrevEdgeEnd
1 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
(22.3121182, 113.930067) NodeLonLat
1 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
1 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
[2290, 2289]
(2542, 2289, 2290, 137.72245475253408, 'gate', 22.3132837, 113.9296113, 22.3121182, 113.930067, (22.312565461414646, 113.92989212481626), 51.77996134806917) STARTEDGE
(2542, 2289, 2290, 137.72245475253408, 'gate', 22.3132837, 113.9296113, 22.3121182, 113.930067, (22.312565461414646, 113.92989212481626), 51.77996134806917) START EDGE (2542, 2289, 2290, 137.72245475253408, 'gate', 22.3132837, 113.9296113, 22.3121182, 113.930067, (22.31321279783337, 113.92963902210839), 2.2524375102628767e-05) LAST EDGE
PREVIOUS EDGE CHOSENNN



(3, (22.312325, 113.93), 'NEW NODE'

1 Connectivity 2290 PrevEdgeEnd
This is the previous edge yo
1 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
1 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
(22.3121182, 113.930067) NodeLonLat
1 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
1 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
0 Connectivity 2290 PrevEdgeEnd
False
True
False
[2290, 775]
(2024, 775, 2290, 25.75088156463692, 'taxiway', 22.3121951, 113.9303033, 22.3121182, 113.930067, (22.31216399609301, 113.93020772323507), 70.00965666218138) STARTEDGE
(2024, 775, 2290, 25.75088156463692, 'taxiway', 22.3121951, 113.9303033, 22.3121182, 113.930067, (22.31216399609301, 113.93020772323507), 70.00965666218138) START EDGE (2542, 2289, 2290, 137.72245475253408, 'gate', 22.3132837, 

0 Connectivity 775 PrevEdgeEnd
2021 1509 1508 49.50229130094012 taxiway 22.3125913 113.9314899 22.3124476 113.9310341 (22.3124476, 113.9310341) 4.5450095769168986
NEW EDGE (2026, 2288, 761, 56.76826257920651, 'taxiway', 22.3119587, 113.9295773, 22.3117891, 113.9290564)
HAVERSINE 107.52365545318888
[775, 2290, 2288]
119.03982050562257 Distance Error! 2288 NewEdgeEnd
0 Connectivity 775 PrevEdgeEnd
2026 2288 761 56.76826257920651 taxiway 22.3119587 113.9295773 22.3117891 113.9290564 (22.3119587, 113.9295773) 5.333333333333333
NEW EDGE (1393, 2397, 764, 51.18894974114103, 'taxiway', 22.3114987, 113.9309801, 22.3119335, 113.9308156)
HAVERSINE 37.38790644374909
[775, 773, 764, 2397]
134.42874910571865 Distance Error! 2397 NewEdgeEnd
0 Connectivity 775 PrevEdgeEnd
1393 2397 764 51.18894974114103 taxiway 22.3114987 113.9309801 22.3119335 113.9308156 (22.3119335, 113.9308156) 4.403781325014727
CHECKING THE NEXT NODE 

0 Connectivity 775 PrevEdgeEnd
0 Connectivity 775 PrevEdgeEnd
1 Connectivity 

1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
This is the previous edge yo
1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
CHECKING THE NEXTNEXT NODE 

1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
1 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity 2397 PrevEdgeEnd
0 Connectivity

...

HAVERSINE 114.151472858652
[1245, 1247, 1242, 10, 1951]
275.44619139241973 Distance Error! 1951 NewEdgeEnd
0 Connectivity 1245 PrevEdgeEnd
962 1951 10 28.676261303874234 taxiway 22.2952903 113.8984765 22.295528 113.8983679 (22.295528, 113.8983679) 5.333333333333333
NEW EDGE (1573, 1242, 1247, 27.80166975421572, 'taxiway', 22.2964615, 113.898793, 22.2965757, 113.8990336)
HAVERSINE 1.7907208521539129
[1245, 1247, 1242]
47.08420969307568 Distance Error! 1247 NewEdgeEnd
1 Connectivity 1245 PrevEdgeEnd
1573 1242 1247 27.80166975421572 taxiway 22.2964615 113.898793 22.2965757 113.8990336 (22.2964757409477, 113.89882300325759) 50.33126870624731
NEW EDGE (963, 10, 1952, 87.4356510940375, 'taxiway', 22.295528, 113.8983679, 22.2962729, 113.8980939)
HAVERSINE 78.5532281441069
[1245, 1247, 1242, 10]
284.096336054561 Distance Error! 10 NewEdgeEnd
0 Connectivity 1245 PrevEdgeEnd
963 10 1952 87.4356510940375 taxiway 22.295528 113.8983679 22.2962729 113.8980939 (22.296200088506993, 113.89812068258705)

EDGE DISCARDED
EDGE DISCARDED
EDGE DISCARDED
EDGE DISCARDED
EDGE DISCARDED
EDGE DISCARDED
EDGE DISCARDED
False
True
False
[1247, 1245, 22, 1947, 39, 41, 44, 46, 47, 49, 53, 55, 411, 438, 52, 43, 1253]
(9155, 1253, 43, 44.3689030962, 'taxiway', 22.2977211, 113.9025339, 22.2981028, 113.9026214, (22.2977211, 113.9025339), 5.333333349882383) STARTEDGE
(9155, 1253, 43, 44.3689030962, 'taxiway', 22.2977211, 113.9025339, 22.2981028, 113.9026214, (22.2977211, 113.9025339), 5.333333349882383) START EDGE (1573, 1242, 1247, 27.80166975421572, 'taxiway', 22.2964615, 113.898793, 22.2965757, 113.8990336, (22.2965757, 113.8990336), 22.74293827720136) LAST EDGE



(96, (22.296844, 113.90001), 'NEW NODE')
22.296535 113.89907 22.2977211 113.9025339 22.2981028 113.9026214
ALPHA 58.89182439607657 BETA 51.352789149317914 D 0.3022841590520539 Theta 0.5172234710019126 Z 53.33333240003994
PREVIOUS EDGE CHOSEN (9155, 1253, 43, 44.3689030962, 'taxiway', 22.2977211, 113.9025339, 22.2981028, 113.9026214, (22.2977

2161 1896 113 12.27328474507383 taxiway 22.3019441 113.9157537 22.3019958 113.9158592 (22.3019441, 113.9157537) 4.057076631395268
NEW EDGE (127, 94, 106, 63.62860856888003, 'taxiway', 22.3009192, 113.9157408, 22.3014569, 113.915528)
HAVERSINE 41.59945977135478
[86, 77, 65, 61, 58, 50, 1836, 1813, 1821, 1831, 2518, 2517, 2516, 2515, 1662, 1663, 2514, 1959, 1587, 1592, 94]
1152.1469820362606 Distance Error! 94 NewEdgeEnd
0 Connectivity 86 PrevEdgeEnd
127 94 106 63.62860856888003 taxiway 22.3009192 113.9157408 22.3014569 113.915528 (22.3014569, 113.915528) 4.009038072736073
NEW EDGE (159, 102, 1970, 81.90385149339174, 'taxiway', 22.3012369, 113.9132749, 22.3014126, 113.9140486)
HAVERSINE 139.29860859274697
[86, 77, 65, 61, 58, 50, 1836, 1813, 1821, 1831, 2518, 2517, 2516, 2515, 1662, 1663, 2514, 1959, 1587, 1592, 94, 106, 1960, 1615, 1315, 1317, 119, 2540, 2539, 2538, 2537, 2536, 1314, 2535, 2534, 1969, 109, 108, 2554, 1307, 104, 101, 100, 99, 102]
2310.7739653113654 Distance Error! 1970 

NEW EDGE (722, 589, 550, 58.20555023648049, 'taxiway', 22.308049, 113.9314315, 22.3075498, 113.931603)
HAVERSINE 100.7446988821913
[505, 541, 543, 581, 2548, 2549, 2119, 2129, 2134, 2135, 603, 589]
396.35255373457505 Distance Error! 550 NewEdgeEnd
0 Connectivity 505 PrevEdgeEnd
722 589 550 58.20555023648049 taxiway 22.308049 113.9314315 22.3075498 113.931603 (22.3075498, 113.931603) 5.333333333333333
NEW EDGE (465, 537, 530, 23.258765735805166, 'taxiway', 22.3072987, 113.9322381, 22.3072155, 113.9320305)
HAVERSINE 32.182549849644815
[505, 541, 543, 581, 2548, 2549, 2119, 2129, 626, 633, 1855, 1847, 621, 2154, 615, 602, 573, 537]
654.7689819239424 Distance Error! 537 NewEdgeEnd
0 Connectivity 505 PrevEdgeEnd
465 537 530 23.258765735805166 taxiway 22.3072987 113.9322381 22.3072155 113.9320305 (22.3072987, 113.9322381) 6.046354921781313
NEW EDGE (2364, 2145, 2146, 29.245308086841586, 'taxiway', 22.3071747, 113.9311569, 22.3073429, 113.9313757)
HAVERSINE 120.86500248336785
[505, 541, 543, 

ValueError: max() arg is an empty sequence

In [ ]:
EdgePath;

In [16]:
FinalNode

[(22.312867565809288, 113.92977400471962),
 (22.312565461414646, 113.92989212481626),
 (22.312118288193417, 113.93006727100266),
 (22.3121182, 113.930067),
 (22.31216399609301, 113.93020772323507),
 (22.3119335, 113.9308156),
 (22.311743979354038, 113.9308873022683),
 (22.311526799146936, 113.930969469113),
 (22.311290339033476, 113.93105882742927),
 (22.311048943115928, 113.93115013703098),
 (22.3107485, 113.9312638),
 (22.3102612, 113.9314391),
 (22.310247696562186, 113.93144435748604),
 (22.3098593, 113.931611),
 (22.309676804405136, 113.93170165828698),
 (22.3089555, 113.9316988),
 (22.309108803731473, 113.93174035235069),
 (22.3088901, 113.9316063),
 (22.308825271591, 113.93137880873611),
 (22.3086067, 113.9306695),
 (22.3086067, 113.9306695),
 (22.308522185829908, 113.93040736615673),
 (22.308416110688757, 113.93007835760827),
 (22.30814763333333, 113.92924563333332),
 (22.30805455158854, 113.92895692582458),
 (22.307918990465854, 113.92853646191836),
 (22.307436125000002, 113.92

In [17]:
from ipynb.fs.full.Results import GPSNodeAdjust, FinalNodeAdjust

True


In [18]:
KMLNode = FinalNodeAdjust(FinalNode)
KMLGPSNode = GPSNodeAdjust(Node)

In [19]:
import simplekml

In [20]:
kml = simplekml.Kml(name="Tracks", open=1)

#Raw Tracks
linestring = kml.newlinestring(name="Raw Tracks")
linestring.coords = KMLGPSNode
linestring.altitudemode = simplekml.AltitudeMode.relativetoground
#Styling
linestring.stylemap.normalstyle.linestyle.color = '9914F0FF'
linestring.stylemap.normalstyle.linestyle.width = 8

linestring.stylemap.highlightstyle.iconstyle.scale = 1.2
linestring.stylemap.highlightstyle.linestyle.color = '9914F0FF'
linestring.stylemap.highlightstyle.linestyle.width = 8

    #linestring.style.linestyle.color = '5014F0FF'  #Yellow
    #linestring.style.linestyle.width = 3  # 3 pixels
    
#Snapped Tracks
linestring = kml.newlinestring(name="Snapped Tracks")
linestring.coords = KMLNode
linestring.altitudemode = simplekml.AltitudeMode.relativetoground
#Edges?
# Styling
linestring.stylemap.normalstyle.linestyle.color = '99F0FF14'
linestring.stylemap.normalstyle.linestyle.width = 8

linestring.stylemap.highlightstyle.iconstyle.scale = 1.2
linestring.stylemap.highlightstyle.linestyle.color = '99F0FF14'
linestring.stylemap.highlightstyle.linestyle.width = 8

    #linestring.style.linestyle.color = simplekml.Color.blue  #Blue
    #linestring.style.linestyle.width = 3  # 3 pixels

kml.save("Flight2.kml")